In [ ]:
import numpy as np

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.transformations.hierarchical.aggregate import Aggregator

# from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.transformations.hierarchical.reconcile import _get_s_matrix
from sktime.utils._testing.hierarchical import _bottom_hier_datagen

In [ ]:
agg = Aggregator(flatten_single_levels=True)

y = _bottom_hier_datagen(
    no_bottom_nodes=3,
    no_levels=1,
    random_seed=111,
)
# add aggregate levels
y = agg.fit_transform(y)

# forecast all levels
fh = ForecastingHorizon([1, 2], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
forecaster.fit(y)

In [ ]:
# fh_resid = ForecastingHorizon(
#     y.index.get_level_values(-1).unique(), is_relative=False
# )
# bug
# forecaster.predict_residuals()

In [ ]:
fh = ForecastingHorizon(y.index.get_level_values(-1).unique(), is_relative=False)
resid = y - forecaster.predict(fh=fh)
resid

In [ ]:
np.arange(resid.index.nlevels).tolist()
grp_range = np.arange(resid.index.nlevels - 1).tolist()
resid = resid.groupby(level=grp_range).apply(
    # lambda x: (x-x.mean())/(x.var()**(1/2))
    lambda x: x
    - x.mean()
)

resid

In [ ]:
resid_mat = resid.unstack().transpose()
print(resid_mat.mean(axis=0))
resid_mat

In [ ]:
cov_mat = resid_mat.transpose().dot(resid_mat) / (len(resid_mat.index) - 1)
cov_mat

In [ ]:
resid_mat.cov()

In [ ]:
import pandas as pd
from numpy.linalg import inv

In [ ]:
def _get_g_matrix_mint(resid_cov):
    """ """
    smat = _get_s_matrix(y)

    g_wls_str = pd.DataFrame(
        np.dot(
            inv(np.dot(np.transpose(smat), np.dot(resid_cov, smat))),
            np.dot(np.transpose(smat), resid_cov),
        )
    )
    # set indexes of matrix
    g_wls_str = g_wls_str.transpose()
    g_wls_str = g_wls_str.set_index(smat.index)
    g_wls_str.columns = smat.columns
    g_wls_str = g_wls_str.transpose()

    return g_wls_str

In [ ]:
gmat = _get_g_matrix_mint(cov_mat)
gmat

In [ ]:
smat = _get_s_matrix(y)

In [ ]:
fh = ForecastingHorizon([1, 2], is_relative=True)
prds = forecaster.predict(fh)
prds

In [ ]:
X = prds.groupby(level=-1)
# could use X.transform() with np.dot, v. marginally faster in my tests
# - loop can use index matching via df.dot() which is probably worth it
recon_preds = []
for _name, group in X:

    # reconcile via SGy
    fcst = smat.dot(gmat.dot(group.droplevel(-1)))
    # add back in time index
    fcst.index = group.index
    recon_preds.append(fcst)

recon_preds = pd.concat(recon_preds, axis=0)
recon_preds = recon_preds.sort_index()
recon_preds

In [ ]:
from sktime.forecasting.reconcile import ReconcilerForecaster

y

In [ ]:
fh = ForecastingHorizon([1, 2], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
recon_forecaster = ReconcilerForecaster(forecaster)
recon_forecaster.fit(y).predict(fh=fh)
# recon_forecaster.fit(y)
# prds = recon_forecaster.predict(fh)

In [ ]:
fh = ForecastingHorizon([1, 2], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
recon_forecaster = ReconcilerForecaster(forecaster)
recon_forecaster.fit_predict(y=y, fh=fh)

In [ ]:
# forecaster._y.reconstruct(forecaster._y)
# forecaster.clone()

recon_forecaster.g_matrix_

In [ ]:
recon_forecaster.predict(fh=fh)

In [ ]:
forecaster._y_in